In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import requests


# Set Download Path
vfdb_setB_url = "http://www.mgc.ac.cn/VFs/Down/VFDB_setB_pro.fas.gz"
drive_folder = "/content/drive/MyDrive/VFDB_Downloads"
local_filename = os.path.join(drive_folder, "VFDB_setB_pro.fas.gz")

# Create the folder if it doesn't exist
os.makedirs(drive_folder, exist_ok=True)

# Download if not already exists
if not os.path.exists(local_filename):
    print(f"Downloading VFDB Set B (Full) into: {local_filename}")
    response = requests.get(vfdb_setB_url)
    with open(local_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {local_filename}")
else:
    print(f"File already exists: {local_filename}")


✅ Downloaded /content/drive/MyDrive/VFDB_Downloads/VFDB_setB_pro.fas.gz


In [13]:
import gzip
import os
import re
import pandas as pd

# 1. Define path
vfdb_setB_path = "/content/drive/MyDrive/VFDB_Downloads/VFDB_setB_pro.fas.gz"

# 2. Extract RefSeq IDs
refseq_ids = []

print(f"Parsing {vfdb_setB_path} for RefSeq IDs...")

with gzip.open(vfdb_setB_path, 'rt') as f:
    for line in f:
        if line.startswith('>'):
            header = line.strip()
            match_refseq = re.search(r'\b(WP_[0-9\.]+)\b', header)
            if match_refseq:
                refseq_ids.append(match_refseq.group(1))

# 3. Save RefSeq IDs
output_dir = "/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs"
os.makedirs(output_dir, exist_ok=True)

if refseq_ids:
    refseq_df = pd.DataFrame(sorted(set(refseq_ids)), columns=["RefSeq_ID"])
    refseq_df.to_csv(os.path.join(output_dir, "vfdb_all_refseq_ids.csv"), index=False)
    print(f"Saved {len(set(refseq_ids))} RefSeq IDs.")
else:
    print("No RefSeq IDs found.")


Parsing /content/drive/MyDrive/VFDB_Downloads/VFDB_setB_pro.fas.gz for RefSeq IDs...
✅ Saved 25141 RefSeq IDs.


In [15]:

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_all_refseq_ids.csv")
print(df.head())
print(df.columns)


      RefSeq_ID
0  WP_000004104
1  WP_000004827
2  WP_000004851
3  WP_000005080
4  WP_000006206
Index(['RefSeq_ID'], dtype='object')


In [30]:
# 1. Download UniProt mapping file
!wget -O /content/idmapping_selected.tab.gz https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz

# 2. Decompress
!gunzip /content/idmapping_selected.tab.gz

# 3. Suppose your RefSeq IDs are in refseq_ids.txt (one ID per line)
!cut -d',' -f1 /content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_all_refseq_ids.csv | tail -n +2 > /content/refseq_ids.txt


--2025-04-27 19:27:00--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/idmapping_selected.tab.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12459633576 (12G) [application/x-gzip]
Saving to: ‘/content/idmapping_selected.tab.gz’

/content/idmapping_ 100%[===================>]  11.60G   108MB/s    in 2m 4s   

2025-04-27 19:29:04 (96.0 MB/s) - ‘/content/idmapping_selected.tab.gz’ saved [12459633576/12459633576]



In [41]:
# Preview the CSV
import pandas as pd

df = pd.read_csv("/content/matched_lines.txt")
print(df.head())
print(df.columns)

  P19361\t28KD_MYCLE\t\tNP_301189.1; WP_010907514.1\t15826926; 499209974; 112728\t\t\tUniRef100_P19361\tUniRef90_P19361\tUniRef50_P19361\tUPI0000124E8E\tA30557\t272631\t\t\t3058804; 11234002\tM23232; AL583917\tAAA25345.1; CAC29599.1\t\t\t\t
0  O52956\tA85A_MYCAV\t75268132\tWP_011723409.1\t...                                                                                                                                                                                            
1  Q05861\tA85A_MYCLE\t\tNP_301195.1; WP_01090752...                                                                                                                                                                                            
2  P58248\tA85A_MYCUL\t\tWP_011742456.1\t14330981...                                                                                                                                                                                            
3  A1KJU9\tA85B_MYCBP\t\tWP_01179922

In [44]:
import pandas as pd

raw_vfdb_path = "/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_all_refseq_ids.csv"
raw_vfdb_df = pd.read_csv(raw_vfdb_path, sep="\t", header=None)

print("Number of columns:", raw_vfdb_df.shape[1])
print("\nFirst few rows:")
print(raw_vfdb_df.head())


🧩 Number of columns: 1

🛠 First few rows:
              0
0     RefSeq_ID
1  WP_000004104
2  WP_000004827
3  WP_000004851
4  WP_000005080


In [45]:
import pandas as pd

# 1. Load the VFDB RefSeq IDs file (skip header automatically)
raw_vfdb_path = "/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_all_refseq_ids.csv"
vfdb_refseq_df = pd.read_csv(raw_vfdb_path)

# 2. Extract unique RefSeq IDs (excluding NaNs)
refseq_ids = vfdb_refseq_df["RefSeq_ID"].dropna().unique().tolist()

print(f"Found {len(refseq_ids)} RefSeq IDs.")

# 3. Save clean RefSeq IDs to a file 
with open("/content/refseq_ids.txt", "w") as f:
    for refseq_id in refseq_ids:
        f.write(refseq_id.strip() + "\n")

print("Saved clean RefSeq IDs to /content/refseq_ids.txt")


✅ Found 25141 RefSeq IDs.
✅ Saved clean RefSeq IDs to /content/refseq_ids.txt


In [46]:
!grep -Ff /content/refseq_ids.txt /content/idmapping_selected.tab > /content/matched_lines.txt


In [47]:

# Load the matched lines
matched_df = pd.read_csv("/content/matched_lines.txt", sep="\t", header=None, names=["UniProtKB_AC", "Database", "RefSeq_ID"])

# Filter only RefSeq mappings
matched_df = matched_df[matched_df["Database"] == "RefSeq"]

# Save the clean mapping
output_csv = "/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_mapped_uniprot_ids.csv"
matched_df[["RefSeq_ID", "UniProtKB_AC"]].drop_duplicates().to_csv(output_csv, index=False)

print(f"✅ Mapping complete! Saved to {output_csv}")


<ipython-input-47-2b93d7533ca7>:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  matched_df = pd.read_csv("/content/matched_lines.txt", sep="\t", header=None, names=["UniProtKB_AC", "Database", "RefSeq_ID"])


✅ Mapping complete! Saved to /content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_mapped_uniprot_ids.csv


In [50]:
!head /content/refseq_ids.txt


WP_000004104
WP_000004827
WP_000004851
WP_000005080
WP_000006206
WP_000006207
WP_000006208
WP_000006209
WP_000006211
WP_000006213


In [53]:
!head matched_lines.txt


P19361	28KD_MYCLE		NP_301189.1; WP_010907514.1	15826926; 499209974; 112728			UniRef100_P19361	UniRef90_P19361	UniRef50_P19361	UPI0000124E8E	A30557	272631			3058804; 11234002	M23232; AL583917	AAA25345.1; CAC29599.1				
O52956	A85A_MYCAV	75268132	WP_011723409.1	13431272; 500042691		GO:0005737; GO:0005576; GO:0004144; GO:0050348	UniRef100_O52956	UniRef90_O52956	UniRef50_P9WQN9	UPI0000125055		1764			9284137	D78144	BAA24156.1				
Q05861	A85A_MYCLE		NP_301195.1; WP_010907520.1	287920; 499209980; 15826932; 603806; 2506968		GO:0005737; GO:0005576; GO:0004144; GO:0050348	UniRef100_Q05861	UniRef90_Q05861	UniRef50_P9WQN9	UPI0000125057	A86921; S32107	272631			7829901; 11234002; 8359887	D43841; M90648; AL583917; Z21950	BAA07864.1; AAA91864.1; CAC29605.1; CAA79948.1				
P58248	A85A_MYCUL		WP_011742456.1	14330981; 18202281; 500062539		GO:0005737; GO:0005576; GO:0004144; GO:0050348	UniRef100_P58248	UniRef90_P58248	UniRef50_P9WQN9	UPI0000125059		1809				AJ300576	CAC40861.1				
A1KJU9	A85B_MYCBP		WP_011

In [54]:
import pandas as pd

# Load the matched lines (they are tab-separated, many fields)
matched_df = pd.read_csv("/content/matched_lines.txt", sep="\t", header=None)

print(f"✅ Matched lines loaded. Columns: {matched_df.shape[1]}")

# Column 0 = UniProtKB AC
# Column 3 = RefSeq IDs (sometimes multiple, separated by ';')

uniprot_ids = []
refseq_to_uniprot = []

for idx, row in matched_df.iterrows():
    uniprot_ac = row[0]
    refseq_field = row[3] if len(row) > 3 else None

    if pd.notna(refseq_field):
        refseq_ids = [rid.strip() for rid in str(refseq_field).split(";")]
        for refseq_id in refseq_ids:
            if refseq_id.startswith(("WP_", "NP_")):
                refseq_to_uniprot.append({"RefSeq_ID": refseq_id, "UniProtKB_AC": uniprot_ac})
                uniprot_ids.append(uniprot_ac)

# Save the clean mapping
output_csv = "/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_mapped_uniprot_ids.csv"
mapping_df = pd.DataFrame(refseq_to_uniprot).drop_duplicates()
mapping_df.to_csv(output_csv, index=False)

print(f"Extracted {len(mapping_df)} RefSeq-UniProt mappings.")
print(f"Unique UniProt IDs: {len(set(uniprot_ids))}")

# Print first 10 UniProt IDs
print("\n🧩 First 10 UniProt IDs:")
print(list(set(uniprot_ids))[:10])


<ipython-input-54-d90bb1d5dec0>:4: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  matched_df = pd.read_csv("/content/matched_lines.txt", sep="\t", header=None)


✅ Matched lines loaded. Columns: 22
✅ Extracted 44202 RefSeq-UniProt mappings.
✅ Unique UniProt IDs: 41999

🧩 First 10 UniProt IDs:
['A0AAD0P5P7', 'J9WIJ3', 'Q1RFV9', 'A0A5I0THM4', 'Q92D47', 'A0A0F7X4Q0', 'A0AAX1J088', 'A6V847', 'Q5XMK7', 'A0A5U4ZX51']


In [56]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1. Load the clean UniProt IDs
mapping_df = pd.read_csv("/content/drive/MyDrive/VFDB_Downloads/VFDB_Extracted_IDs/vfdb_mapped_uniprot_ids.csv")
uniprot_ids = mapping_df["UniProtKB_AC"].dropna().unique().tolist()

print(f"Found {len(uniprot_ids)} UniProt IDs to download.")

# 2. Set output folder
output_folder = "/content/drive/MyDrive/VFDB_Downloads/VFDB_PDBs"
os.makedirs(output_folder, exist_ok=True)

# 3. Define download function
def download_alphafold_pdb(uniprot_id):
    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
    output_path = os.path.join(output_folder, f"{uniprot_id}.pdb")

    if os.path.exists(output_path):
        return {"UniProt_ID": uniprot_id, "Status": "Already exists"}

    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            with open(output_path, "wb") as f:
                f.write(response.content)
            return {"UniProt_ID": uniprot_id, "Status": "Downloaded"}
        else:
            return {"UniProt_ID": uniprot_id, "Status": "Not found"}
    except Exception as e:
        return {"UniProt_ID": uniprot_id, "Status": f"Error: {e}"}

# 4. Multi-threaded download
download_summary = []

max_workers = 8

print(f"🚀 Starting parallel downloads with {max_workers} workers...")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(download_alphafold_pdb, uid): uid for uid in uniprot_ids}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading AlphaFold2 PDBs (fast)"):
        result = future.result()
        download_summary.append(result)

# 5. Save download summary
summary_df = pd.DataFrame(download_summary)
summary_csv = os.path.join(output_folder, "download_summary.csv")
summary_df.to_csv(summary_csv, index=False)

print(f"\nDownload complete! Summary saved at {summary_csv}")

# 6. Print download stats
print("\nDownload Summary:")
if not summary_df.empty:
    print(summary_df['Status'].value_counts())
else:
    print("No downloads attempted — check UniProt ID list!")


✅ Found 41999 UniProt IDs to download.
🚀 Starting parallel downloads with 20 workers...



✅ Download complete! Summary saved at /content/drive/MyDrive/VFDB_Downloads/VFDB_PDBs/download_summary.csv

📋 Download Summary:
Status
Downloaded        30648
Not found          8340
Already exists     3011
Name: count, dtype: int64


In [2]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
from Bio.PDB import PDBParser, DSSP
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

# 1. Paths
pdb_folder = "/content/drive/MyDrive/VFDB_Downloads/VFDB_PDBs"
output_csv = "/content/drive/MyDrive/VFDB_Downloads/VFDB_PDBs/structure_features_extracted_clean.csv"

parser = PDBParser(QUIET=True)

# 2. Feature extraction function
def extract_features_optimized(pdb_path):
    features = {
        "UniProt_ID": os.path.basename(pdb_path).split(".")[0],
        "mean_pLDDT": np.nan,
        "alpha_helix_content": np.nan,
        "beta_sheet_content": np.nan,
        "coil_content": np.nan,
        "hbond_density": np.nan
    }
    try:
        structure = parser.get_structure("model", pdb_path)
        model = structure[0]

        # Mean pLDDT
        b_factors = [atom.get_bfactor() for atom in model.get_atoms()]
        if b_factors:
            features["mean_pLDDT"] = np.mean(b_factors)

        # DSSP 
        dssp = DSSP(model, pdb_path, acc_array='Wilke')

        ss_counts = {"H":0, "B":0, "E":0, "G":0, "I":0, "T":0, "S":0, "-":0}
        hbonds = []

        for key in dssp.keys():
            ss = dssp[key][2]
            if ss in ss_counts:
                ss_counts[ss] += 1
            hbonds.append(dssp[key][3])

        total = sum(ss_counts.values())
        if total > 0:
            helix_types = ss_counts["H"] + ss_counts["G"] + ss_counts["I"]
            sheet_types = ss_counts["E"] + ss_counts["B"]
            coil_types = ss_counts["T"] + ss_counts["S"] + ss_counts["-"]

            features["alpha_helix_content"] = helix_types / total
            features["beta_sheet_content"] = sheet_types / total
            features["coil_content"] = coil_types / total

        if hbonds:
            features["hbond_density"] = np.mean(hbonds)

    except Exception as e:
        pass

    return features

# 3. List PDB files
pdb_files = [os.path.join(pdb_folder, f) for f in os.listdir(pdb_folder) if f.endswith(".pdb")]
print(f"✅ Found {len(pdb_files)} PDB files.")

# 4. Prepare CSV and header
columns = ["UniProt_ID", "mean_pLDDT", "alpha_helix_content", "beta_sheet_content", "coil_content", "hbond_density"]
pd.DataFrame(columns=columns).to_csv(output_csv, index=False)

# 5. Start parallel feature extraction (buffered)
max_workers = 12  # Or however many CPUs you found
print(f"Starting smarter DSSP feature extraction with {max_workers} workers... (Buffered partial saves)")

buffer = []
buffer_size = 1000  # Save every 100 entries
counter = 0

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(extract_features_optimized, pdb_path): pdb_path for pdb_path in pdb_files}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Extracting Features (Buffered Save Mode)"):
        result = future.result()

        if result:
            buffer.append(result)
            counter += 1

            if counter >= buffer_size:
                pd.DataFrame(buffer).to_csv(output_csv, mode='a', header=False, index=False)
                buffer = []
                counter = 0

# Save any leftover features at the end
if buffer:
    pd.DataFrame(buffer).to_csv(output_csv, mode='a', header=False, index=False)

print(f"\nFeature extraction complete! Saved to {output_csv}")


✅ Found 33659 PDB files.
🚀 Starting smarter DSSP feature extraction with 12 workers... (Buffered partial saves)


Extracting Features (Buffered Save Mode):   1%|          | 369/33659 [01:11<1:47:21,  5.17it/s]
